In [2]:
# %load C:\Users\Walter\Desktop\tools\template\plot_template.py

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

plt.rc('font', family='serif')
plt.rc('axes', labelsize=16)
plt.rc('xtick', labelsize=14, color='grey')
plt.rc('ytick', labelsize=14, color='grey')
plt.rc('legend', fontsize=16, loc='lower left')
plt.rc('savefig', dpi=330, bbox='tight')
%matplotlib inline


from sklearn.linear_model import LinearRegression

## Define utility functions
#### Read data

In [38]:
def read_chiller_data(filename):
    file = 'data/' + filename + '.csv'
    data = pd.read_csv(file,index_col=0)
    if 'Excel Time' in df.columns:
        df.drop('Excel Time',axis=1,inplace=True)
    data.index = pd.to_datetime(data.index)
    data.rename(columns={'chwr':'cw_e','chws':'cw_l','CW Flow':'cond_f',
                         'CWR Temp':'cond_l','CWS Temp':'cond_e','Load':'load','MOTOR KW':'power'},inplace=True)
    return data

#### Data quality check

In [ ]:
def load_error_distribution(data, savefig=False, filename='test'):
    df = data[data['on']]
    df['heat_rejection'] = df['cond_f']*(df['cond_l']-df['cond_e'])*4200
    df['error'] = df['heat_rejection']/(df['load']+df['power'])-1
    # plot
    plt.figure(figsize=(7,5))
    plt.hist(df['error'],bins=[i/10 for i in range(-6,7)],color='skyblue', label='reference model')
    plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    plt.xlabel('Heat imbalance rate')
    plt.xlim(-0.6,0.6)
    plt.ylabel('Number of measurements')
    if savefig:
        plt.savefig('fig/'+test+'.png')
    return None

## Prepare the data for regression

In [13]:
def IP_to_SI(data):
    df = data.copy()   # !important, to avoid contaminating the raw data
    # convert degF to degC
    for field in ['cw_e','cw_l','cond_e','cond_l']:
        df[field] = (df[field]-32)*5/9
    # convert RT (Refrigeration Tons) to kW
    df['load'] = df['load']*3.517
    # convert gpm to m3/s
    df['cond_f'] = df['cond_f']*6.31*10**(-5)
    return df

def prepare_data(chiller_name,threshold=0.2):
    '''
    input 
    chiller_name: string
    threshold: heat rejected to the CT should be equal to the sum of cooling load and cooling power, using this criteria to filter out bad data
               After the filtering, the load_balance should be in the range [-threshold,threshold] 
    '''
    data_IP = read_chiller_data(chiller_name)
    
    # convert the unit from IP to SI
    data_SI = IP_to_SI(data_IP)
    # select the data point when the chiller is on
    data_SI_on = data_SI[data_SI.power>100]
    # filter out data with large heat balance error
    data_SI_on['heat_rejection'] = data_SI_on['cond_f']*(data_SI_on['cond_l']-data_SI_on['cond_e'])*4200
    data_SI_on['error'] = data_SI_on['heat_rejection']/(data_SI_on['load']+data_SI_on['power'])-1
    flag_bad_data = (np.abs(data_SI_on['error']) > threshold)
    print('filtering out {} data points according to the heat balance criterion'.format(sum(flag_bad_data)/data_SI_on.shape[0]))
    data_SI_on = data_SI_on[~ flag_bad_data]  
    
    return data_SI_on

## Define the regression function

Reference: DOE chiller model, E+ engineering reference Page 767

In [24]:
def chillerCapFTemp(data):
    df_op = data.loc[:,['cw_l','cond_e','load']].dropna()
    df_op['cw_l_sq'] = df_op['cw_l']**2
    df_op['cond_e_sq'] = df_op['cond_e']**2
    df_op['cw_l*cond_e'] = df_op['cw_l']*df_op['cond_e']
    y = df_op.pop('load').values
    x = df_op.values
    model = LinearRegression()
    model.fit(x,y)
    r_sq = model.score(x, y)
    print('coefficient of determination:', r_sq)
    result = {'intercept':model.intercept_}
    for i in range(df_op.shape[1]):
        result[df_op.columns[i]] = model.coef_[i]
    result['r_sq'] = r_sq
    return result

In [25]:
def chillerEIRFTemp(data):
    df_op = data.loc[:,['cw_l','cond_e','load','power']].dropna()
    df_op['cw_l_sq'] = df_op['cw_l']**2
    df_op['cond_e_sq'] = df_op['cond_e']**2
    df_op['cw_l*cond_e'] = df_op['cw_l']*df_op['cond_e']
    y = df_op.pop('power')/df_op.pop('load').values         # power to load, inverse of COP
    x = df_op.values
    model = LinearRegression()
    model.fit(x,y)
    r_sq = model.score(x, y)
    print('coefficient of determination:', r_sq)
    result = {'intercept':model.intercept_}
    for i in range(df_op.shape[1]):
        result[df_op.columns[i]] = model.coef_[i]
    result['r_sq'] = r_sq
    return result

## Regression and saving results

In [68]:
coefficient_df_cap = pd.DataFrame()
coefficient_df_EIR = pd.DataFrame()
for i in range(1,6):
    data = prepare_data('chi'+str(i))
    ## regress on the cooling capacity
    coefficient_CapFTemp = chillerCapFTemp(data)
    df_cap = pd.DataFrame(coefficient_CapFTemp, index=[i])
    coefficient_df_cap = pd.concat([coefficient_df_cap,df_cap])
    ## regress on the EIR (cooling efficiency)
    coefficient_EIR = chillerEIRFTemp(data)
    df_EIR = pd.DataFrame(coefficient_EIR, index=[i])
    coefficient_df_EIR = pd.concat([coefficient_df_EIR,df_EIR])

C:\Users\Walter\Anaconda3\lib\site-packages\dateutil\parser\_parser.py:1206: UnknownTimezoneWarning: tzname PDT identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)
C:\Users\Walter\Anaconda3\lib\site-packages\dateutil\parser\_parser.py:1206: UnknownTimezoneWarning: tzname PST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)
C:\Users\Walter\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Walter\Anaconda3\lib\site-pa

filtering out 0.0819081908190819 data points according to the heat balance criterion
coefficient of determination: 0.707549786411378
filtering out 0.12754898040783685 data points according to the heat balance criterion
coefficient of determination: 0.30696888866731875
filtering out 0.2685355957211361 data points according to the heat balance criterion
coefficient of determination: 0.5092301052046538
filtering out 0.6574665541762083 data points according to the heat balance criterion
coefficient of determination: 0.7695376300581731
filtering out 0.02848465807938004 data points according to the heat balance criterion
coefficient of determination: 0.8771563270207381


In [64]:
coefficient_df_cap.T.to_csv('chillerCap.csv')

In [ ]:
coefficient_df_EIR.T.to_csv('chillerEIR.csv')